# Live feed from camera

1. https://opencv.org/ 
1. Install Opencv
1. Live Camera Feed

In [1]:
!pip install opencv-python

### Live Camera Feed

In [3]:
# Import
import cv2

# Access camera
cap = cv2.VideoCapture(0)

# Read/Show frame's from camera
while True:
    _, frame = cap.read()
    
    cv2.imshow('Live', frame)
    if cv2.waitKey(1) == 27: # ESC
        break

cap.release()

# Hand Tracking

1. https://mediapipe.dev/
1. Install Mediapipe
2. Load Tracking Model
3. Implement Live Tracking

In [2]:
# !pip install mediapipe

# For M1 Chip
!pip install mediapipe-silicon

### Import & Load Model

In [1]:
import mediapipe as mp

# Load Model
hands = mp.solutions.hands
hand_landmark = hands.Hands(max_num_hands=1)

objc[18404]: Class CaptureDelegate is implemented in both /Users/diwakargupta/miniconda3/envs/cvpipe/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x104dc8860) and /Users/diwakargupta/miniconda3/envs/cvpipe/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16645e480). One of the two will be used. Which one is undefined.
objc[18404]: Class CVWindow is implemented in both /Users/diwakargupta/miniconda3/envs/cvpipe/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x103dd8a68) and /Users/diwakargupta/miniconda3/envs/cvpipe/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16645e4d0). One of the two will be used. Which one is undefined.
objc[18404]: Class CVView is implemented in both /Users/diwakargupta/miniconda3/envs/cvpipe/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x103dd8a90) and /Users/diwakargupta/miniconda3/envs/cvpipe/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16645e4f8). One of the two w

In [4]:
draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hand_landmark.process(rgb)
    
    if op.multi_hand_landmarks:
        # Draw hand landmarks
        for all_landmarks in op.multi_hand_landmarks:
            draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS) # Explain parameters
    
    cv2.imshow('Live', frame)
    
    if cv2.waitKey(10) == 27:
        break
  
cap.release()
cv2.waitKey(1)

-1

### Hand and Face Tracking

In [13]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5)


cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1) # 0 vertically x-axis, 1 horizontally y-axis
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hand_landmark.process(rgb)
    results = face_mesh.process(rgb)
    
    if op.multi_hand_landmarks:
        all_landmarks = next(iter(op.multi_hand_landmarks))
        draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS)
        
    if results.multi_face_landmarks:
        for i in results.multi_face_landmarks:
            draw.draw_landmarks(frame, i, mp_face_mesh.FACEMESH_IRISES)
    
    cv2.imshow('Live', frame)
    
    if cv2.waitKey(10) == 27:
        break
  
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# Draw

1. Location of index finger
2. Draw Line

In [5]:
# COPY / PASTE

# Camera frame resolution
frame_shape = (720, 1280, 3)

## Index Finger Location

Finger indexes https://google.github.io/mediapipe/solutions/hands

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hand_landmark.process(rgb)
    
    if op.multi_hand_landmarks:
        for all_landmarks in op.multi_hand_landmarks:
            draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS)

            # index finger location
            x = all_landmarks.landmark[8].x * frame_shape[1]
            y = all_landmarks.landmark[8].y * frame_shape[0]
            x, y = int(x), int(y)
    
    cv2.imshow('Live', frame)
    
    if cv2.waitKey(1) == 27:
        break
  
cap.release()
cv2.waitKey(1)

## Draw Line

In [6]:
import numpy as np

mask = np.zeros(frame_shape, dtype='uint8')
colour = (125, 100, 140)
thickness = 4

cap = cv2.VideoCapture(0)
prevxy = None

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hand_landmark.process(rgb)
    
    if op.multi_hand_landmarks:
        for all_landmarks in op.multi_hand_landmarks:
            draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS)

            # index finger location
            x = all_landmarks.landmark[8].x * frame_shape[1]
            y = all_landmarks.landmark[8].y * frame_shape[0]
            x, y = int(x), int(y)
            
            # Connect previous and current index finger locations
            if prevxy!=None:
                cv2.line(mask, prevxy, (x, y), colour, thickness)
            prevxy = (x, y)
    
    # Merge Frame and Mask
    frame = np.where(mask, mask, frame)
    
    cv2.imshow('Live', frame)
    if cv2.waitKey(1) == 27:
        break
  
cap.release()
cv2.waitKey(1)

-1

# Tool Bar

In [7]:
# Read toolbar image

tools = cv2.imread("tool.png")
tools = tools.astype('uint8')
print(tools.shape)

(50, 250, 3)


In [8]:
frame_shape = (720, 1280, 3)

# Row and Column for toolbar
midCol = 1280 // 2
max_row = 50
min_col = midCol-125
max_col = midCol+125

In [10]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1) 
    
    frame[0:max_row, min_col:max_col] = tools
    
    cv2.imshow('Live', frame)
    
    if cv2.waitKey(10) == 27:
        break
  
cap.release()
cv2.waitKey(1)

-1

# Rectangle

Drow only when index finger and middle finger are close enough

In [11]:
# This are some utility functions

# Check if distance between 2 points is less than 60 pixels
def get_is_clicked(point1, point2):
    (x1, y1) = point1
    (x2, y2) = point2
    
    dis = (x1-x2)**2 + (y1-y2)**2
    dis = np.sqrt(dis)
    if dis<60:
        return True
    else:
        return False

# Return tool based on column location
def get_Tool(point, prev_tool):
    (x, y) = point
    
    if x>min_col and x<max_col and y<max_row:
        if x < min_col:
            return
        elif x < 50 + min_col:
            curr_tool = "line"
        elif x<100 + min_col:
            curr_tool = "rectangle"
        elif x < 150 + min_col:
            curr_tool ="draw"
        elif x<200 + min_col:
            curr_tool = "circle"
        else:
            curr_tool = "erase"
        return curr_tool
    else:
        return prev_tool

In [13]:
import numpy as np

mask = np.zeros(frame_shape, dtype='uint8')
colour = (125, 100, 140)
thickness = 4
curr_tool = 'draw'
start_point = None

cap = cv2.VideoCapture(0)
prevxy = None

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    # Preprocess Image
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hand_landmark.process(rgb)
    
    # Check if hand is in frame
    if op.multi_hand_landmarks:
        for all_landmarks in op.multi_hand_landmarks:
            draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS)

            # index finger location
            x = all_landmarks.landmark[8].x * frame_shape[1]
            y = all_landmarks.landmark[8].y * frame_shape[0]
            x, y = int(x), int(y)
            
            # Middle finger location
            middle_x = all_landmarks.landmark[12].x * frame_shape[1]
            middle_y = all_landmarks.landmark[12].y * frame_shape[0]
            middle_x, middle_y = int(middle_x), int(middle_y)
            
            is_clicked = get_is_clicked((x, y), (middle_x, middle_y))
            curr_tool = get_Tool((x, y), curr_tool)

            # Select tool and draw for that
            if curr_tool == 'draw':
                # Connect previous and current index finger locations
                if is_clicked and prevxy!=None:
                    cv2.line(mask, prevxy, (x, y), colour, thickness)
            elif curr_tool == 'rectangle':
                if is_clicked and start_point==None:
                    start_point = (x, y)
                
                elif is_clicked:
                    cv2.rectangle(frame, start_point, (x, y), colour, thickness)
                
                elif is_clicked==False and start_point:
                    cv2.rectangle(mask, start_point, (x, y), colour, thickness)
                    start_point=None
            prevxy = (x, y)    
    
    # Merge Frame and Mask
    frame = np.where(mask, mask, frame)
    
    frame[0:max_row, min_col:max_col] = tools
    cv2.imshow('Live', frame)
    if cv2.waitKey(1) == 27:
        break
  
cap.release()
cv2.waitKey(1)

-1

# Eraser & Circle

In [16]:
import numpy as np

mask = np.zeros(frame_shape, dtype='uint8')
colour = (125, 100, 140)
thickness = 4
curr_tool = 'draw'
start_point = None

cap = cv2.VideoCapture(0)
prevxy = None

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    # Preprocess Image
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = hand_landmark.process(rgb)
    
    # Check if hand is in frame
    if op.multi_hand_landmarks:
        for all_landmarks in op.multi_hand_landmarks:
            draw.draw_landmarks(frame, all_landmarks, hands.HAND_CONNECTIONS)

            # index finger location
            x = all_landmarks.landmark[8].x * frame_shape[1]
            y = all_landmarks.landmark[8].y * frame_shape[0]
            x, y = int(x), int(y)
            
            # Middle finger location
            middle_x = all_landmarks.landmark[12].x * frame_shape[1]
            middle_y = all_landmarks.landmark[12].y * frame_shape[0]
            middle_x, middle_y = int(middle_x), int(middle_y)
            
            is_clicked = get_is_clicked((x, y), (middle_x, middle_y))
            curr_tool = get_Tool((x, y), curr_tool)

            # Select tool and draw for that
            if curr_tool == 'draw':
                # Connect previous and current index finger locations
                if is_clicked and prevxy!=None:
                    cv2.line(mask, prevxy, (x, y), colour, thickness)
            
            elif curr_tool == 'rectangle':
                if is_clicked and start_point==None:
                    # Init start_point
                    start_point = (x, y)
                elif is_clicked:
                    # Draw temp rectange
                    cv2.rectangle(frame, start_point, (x, y), colour, thickness)
                elif is_clicked==False and start_point:
                    # draw perm. rectangle and reset start_point
                    cv2.rectangle(mask, start_point, (x, y), colour, thickness)
                    start_point=None
            
            elif curr_tool=='circle':
                if is_clicked and start_point==None:
                    start_point = (x, y)
                
                if start_point:
                    rad = int(((start_point[0]-x)**2 + (start_point[1]-y)**2)**0.5)
                if is_clicked:
                    cv2.circle(frame, start_point, rad, colour, thickness)
                
                if is_clicked==False and start_point:
                    cv2.circle(mask, start_point, rad, colour, thickness)
                
                    start_point=None
            
            elif curr_tool == "erase":
                cv2.circle(frame, (x, y), 30, (0,0,0), -1) # -1 means fill
                if is_clicked:
                    cv2.circle(mask, (x, y), 30, 0, -1)
            prevxy = (x, y)    
    
    # Merge Frame and Mask
    frame = np.where(mask, mask, frame)
    
    frame[0:max_row, min_col:max_col] = tools
    cv2.imshow('Live', frame)
    if cv2.waitKey(1) == 27:
        break
  
cap.release()
cv2.waitKey(1)

-1